In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
dataframe = pd.read_csv('space_decay.csv')

In [3]:
dataframe.head(2)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T06:46:11,18 SPCS,ARIANE 42P+ DEB,1992-072J,EARTH,TEME,UTC,SGP4,...,MEDIUM,FR,1992.0,FRGUI,NaN,3195178,188614016,0 ARIANE 42P+ DEB,1 26741U 92072J 21304.94919376 .00000883 0...,2 26741 7.7156 90.2410 6528926 243.1216 38...
1,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T04:58:37,18 SPCS,SL-8 DEB,1979-028C,EARTH,TEME,UTC,SGP4,...,SMALL,CIS,1979.0,PKMTR,NaN,3194950,188593285,0 SL-8 DEB,1 26743U 79028C 21304.68908982 .00000079 0...,2 26743 82.9193 299.1120 0030720 158.9093 201...


In [4]:
dataframe.shape

(14372, 40)

In [5]:
dataframe.dtypes

CCSDS_OMM_VERS           int64
COMMENT                 object
CREATION_DATE           object
ORIGINATOR              object
OBJECT_NAME             object
OBJECT_ID               object
CENTER_NAME             object
REF_FRAME               object
TIME_SYSTEM             object
MEAN_ELEMENT_THEORY     object
EPOCH                   object
MEAN_MOTION            float64
ECCENTRICITY           float64
INCLINATION            float64
RA_OF_ASC_NODE         float64
ARG_OF_PERICENTER      float64
MEAN_ANOMALY           float64
EPHEMERIS_TYPE           int64
CLASSIFICATION_TYPE     object
NORAD_CAT_ID             int64
ELEMENT_SET_NO           int64
REV_AT_EPOCH             int64
BSTAR                  float64
MEAN_MOTION_DOT        float64
MEAN_MOTION_DDOT       float64
SEMIMAJOR_AXIS         float64
PERIOD                 float64
APOAPSIS               float64
PERIAPSIS              float64
OBJECT_TYPE             object
RCS_SIZE                object
COUNTRY_CODE            object
LAUNCH_D

In [6]:
dataframe.isna().sum()

CCSDS_OMM_VERS             0
COMMENT                    0
CREATION_DATE              0
ORIGINATOR                 0
OBJECT_NAME                0
OBJECT_ID                 39
CENTER_NAME                0
REF_FRAME                  0
TIME_SYSTEM                0
MEAN_ELEMENT_THEORY        0
EPOCH                      0
MEAN_MOTION                0
ECCENTRICITY               0
INCLINATION                0
RA_OF_ASC_NODE             0
ARG_OF_PERICENTER          0
MEAN_ANOMALY               0
EPHEMERIS_TYPE             0
CLASSIFICATION_TYPE        0
NORAD_CAT_ID               0
ELEMENT_SET_NO             0
REV_AT_EPOCH               0
BSTAR                      0
MEAN_MOTION_DOT            0
MEAN_MOTION_DDOT           0
SEMIMAJOR_AXIS             0
PERIOD                     0
APOAPSIS                   0
PERIAPSIS                  0
OBJECT_TYPE                0
RCS_SIZE                 198
COUNTRY_CODE              39
LAUNCH_DATE               39
SITE                      39
DECAY_DATE    

In [7]:
dataframe = dataframe.drop(columns = ['DECAY_DATE'])

In [8]:
dataframe['RCS_SIZE'].unique()

array(['MEDIUM', 'SMALL', 'LARGE', nan], dtype=object)

In [9]:
dataframe = dataframe[dataframe['RCS_SIZE'].notna()]

In [10]:
dataframe.dtypes

CCSDS_OMM_VERS           int64
COMMENT                 object
CREATION_DATE           object
ORIGINATOR              object
OBJECT_NAME             object
OBJECT_ID               object
CENTER_NAME             object
REF_FRAME               object
TIME_SYSTEM             object
MEAN_ELEMENT_THEORY     object
EPOCH                   object
MEAN_MOTION            float64
ECCENTRICITY           float64
INCLINATION            float64
RA_OF_ASC_NODE         float64
ARG_OF_PERICENTER      float64
MEAN_ANOMALY           float64
EPHEMERIS_TYPE           int64
CLASSIFICATION_TYPE     object
NORAD_CAT_ID             int64
ELEMENT_SET_NO           int64
REV_AT_EPOCH             int64
BSTAR                  float64
MEAN_MOTION_DOT        float64
MEAN_MOTION_DDOT       float64
SEMIMAJOR_AXIS         float64
PERIOD                 float64
APOAPSIS               float64
PERIAPSIS              float64
OBJECT_TYPE             object
RCS_SIZE                object
COUNTRY_CODE            object
LAUNCH_D

In [11]:
    # eccentricity = row['ECCENTRICITY']
    # inclination = row['INCLINATION']
    # semi_major_axis = row['SEMIMAJOR_AXIS']

dataframe[['ECCENTRICITY' , 'INCLINATION' , 'SEMIMAJOR_AXIS']].describe()

,ECCENTRICITY,INCLINATION,SEMIMAJOR_AXIS
count,14174.000000,14174.000000,14174.000000
mean,0.063655,75.050788,10381.409239
std,0.178477,29.024966,8856.388961
min,0.000005,0.001400,6542.850000
25%,0.000701,53.055600,6953.224000
50%,0.003314,86.392850,7174.004000
75%,0.012941,98.578300,7433.195250
max,0.897218,144.586200,182331.558000


https://space.stackexchange.com/questions/23450/determine-orbit-type-from-tle#:~:text=We%20can%20use%20the%20following,35%2C786km).


In [12]:
def categorize_orbit(row):
    eccentricity = row['ECCENTRICITY']
    mean_motion = row['MEAN_MOTION']
    period = row['PERIOD']

    if eccentricity > 0.25:
        return 'HEO'  # Highly Elliptical Orbit
    elif (0.99 <= mean_motion <= 1.01) and eccentricity < 0.01:
        return 'GEO'  # Geostationary Orbit
    elif (600 <= period <= 800) and eccentricity < 0.25:
        return 'MEO'  # Medium Earth Orbit
    elif mean_motion > 11.25 and eccentricity < 0.25:
        return 'LEO'  # Low Earth Orbit
    else:
        return 'Other'

dataframe['ORBIT_TYPE'] = dataframe.apply(categorize_orbit, axis=1)

In [13]:
dataframe['ORBIT_TYPE'].unique()

array(['HEO', 'LEO', 'Other', 'GEO', 'MEO'], dtype=object)

In [14]:
print(dataframe[dataframe['ORBIT_TYPE'] == 'Other'].count().sum())
print(dataframe[dataframe['ORBIT_TYPE'] == 'HEO'].count().sum())
print(dataframe[dataframe['ORBIT_TYPE'] == 'LEO'].count().sum())
print(dataframe[dataframe['ORBIT_TYPE'] == 'GEO'].count().sum())
print(dataframe[dataframe['ORBIT_TYPE'] == 'MEO'].count().sum())

10160
50240
478200
21760
6600


In [15]:
dataframe['CURRENT_YEAR'] = 2023
dataframe["OBJECT_AGE"] = dataframe.CURRENT_YEAR - dataframe.LAUNCH_DATE

dataframe.RCS_SIZE = dataframe.RCS_SIZE.map({'SMALL':1, 'MEDIUM':2, 'LARGE':3})

dataframe['ORBIT_TYPE_MAPPED'] = dataframe.ORBIT_TYPE.map({'Other':0, 'HEO':1, 'LEO':2, 'GEO':3, 'MEO':4})

In [ ]:
dataframe[["OBJECT_AGE","RCS_SIZE","APOAPSIS","PERIAPSIS","MEAN_ANOMALY","MEAN_MOTION",
           "INCLINATION","PERIOD","ECCENTRICITY","ORBIT_TYPE"
           ,"ORBIT_TYPE_MAPPED"]].corr().style.background_gradient()

ValueError: could not convert string to float: 'HEO'

In [ ]:
# numeric_columns = dataframe.select_dtypes(include=['int64', 'float64'])

# numeric_columns = numeric_columns.drop(['ORBIT_TYPE_MAPPED'], axis=1)

# numeric_columns = numeric_columns.abs()

# X = numeric_columns.drop('RCS_SIZE', axis=1)
# y = dataframe['RCS_SIZE']

# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)

# best_features = SelectKBest(score_func=chi2, k='all')
# fit = best_features.fit(X_scaled, y)

# feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': fit.scores_})
# best_features = feature_scores.nlargest(10, 'Score')

# print(best_features)

In [17]:
finalAnalysisColumns = [
"OBJECT_AGE",
"RCS_SIZE",
"APOAPSIS",
"PERIAPSIS",
"MEAN_ANOMALY",
"MEAN_MOTION",
"INCLINATION",
"PERIOD",
"ECCENTRICITY",
"ORBIT_TYPE_MAPPED"]

columnsForSize = ["OBJECT_AGE","APOAPSIS","PERIAPSIS","PERIOD","ECCENTRICITY"]

In [18]:
dataframe = dataframe[finalAnalysisColumns]

In [19]:
dataframe.head()

,OBJECT_AGE,RCS_SIZE,APOAPSIS,PERIAPSIS,MEAN_ANOMALY,MEAN_MOTION,INCLINATION,PERIOD,ECCENTRICITY,ORBIT_TYPE_MAPPED
0,31.0,2,27784.871,796.095,38.7796,2.921700,7.7156,492.864,0.652893,1
1,44.0,1,1002.507,957.299,201.3337,13.754973,82.9193,104.689,0.003072,2
2,22.0,3,35797.696,33841.738,146.2900,1.038224,12.1717,1386.984,0.023739,0
3,24.0,1,679.465,594.417,323.1632,14.775907,98.4781,97.456,0.006062,2
4,24.0,1,697.039,609.491,27.6061,14.724482,98.4232,97.796,0.006226,2


In [20]:
X = dataframe.drop(columns = ['RCS_SIZE','ORBIT_TYPE_MAPPED'], axis=1)
y = dataframe['RCS_SIZE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X

,OBJECT_AGE,APOAPSIS,PERIAPSIS,MEAN_ANOMALY,MEAN_MOTION,INCLINATION,PERIOD,ECCENTRICITY
0,31.0,27784.871,796.095,38.7796,2.921700,7.7156,492.864,0.652893
1,44.0,1002.507,957.299,201.3337,13.754973,82.9193,104.689,0.003072
2,22.0,35797.696,33841.738,146.2900,1.038224,12.1717,1386.984,0.023739
3,24.0,679.465,594.417,323.1632,14.775907,98.4781,97.456,0.006062
4,24.0,697.039,609.491,27.6061,14.724482,98.4232,97.796,0.006226
...,...,...,...,...,...,...,...,...
14295,2.0,35226.188,246.490,209.6533,2.318388,5.9230,621.121,0.725284
14296,2.0,35102.192,236.603,223.6129,2.328075,5.7471,618.537,0.724931
14297,2.0,35904.963,32368.563,132.8297,1.064586,33.9986,1352.639,0.043643
14298,2.0,546.812,532.580,208.2856,15.088562,97.5434,95.437,0.001029


In [34]:
rf_classifier = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Best Parameters:", best_params)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

print("\nTraining Classification Report:")
print(classification_report(y_train, y_train_pred))

print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred))

conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix - Test Set:")
print(conf_matrix_test)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Training Accuracy: 0.9998236176029632
Test Accuracy: 0.9044091710758377

Training Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      6656
           2       1.00      1.00      1.00      1304
           3       1.00      1.00      1.00      3379

    accuracy                           1.00     11339
   macro avg       1.00      1.00      1.00     11339
weighted avg       1.00      1.00      1.00     11339


Test Classification Report:
              precision    recall  f1-score   support

           1       0.93      0.96      0.95      1690
           2       0.72      0.59      0.65       335
           3       0.91      0.92      0.91       810

    accuracy                           0.90      2835
   macro avg       0.85      0.82      0.84      2835
weighted avg       0.90      0.90      0.90      2835


In [ ]:
# joblib.dump(best_model, 'rcs_size_classifier.pkl')

['rcs_size_classifier.pkl']

In [ ]:
# loaded_model = joblib.load('random_forest_model.pkl')

# # 22.0	3	35797.696	33841.738	146.2900	1.038224	12.1717	1386.984	0.023739	0

# input_data = {
#     'OBJECT_AGE': [22.0],  # Replace with your input values
#     'APOAPSIS': [35797.696],
#     'PERIAPSIS': [33841.738],
#     'MEAN_ANOMALY': [146.2900],
#     'MEAN_MOTION': [1.038224],
#     'INCLINATION': [12.1717],
#     'PERIOD': [1386.984],
#     'ECCENTRICITY': [0.023739]
# }

# new_df = pd.DataFrame(input_data)

# size_mapping = {1: 'SMALL', 2: 'MEDIUM', 3: 'LARGE'}

# predictions = loaded_model.predict(new_df)

# # Map numerical predictions to sizes
# predicted_sizes = [size_mapping[pred] for pred in predictions]

# # Display the predicted sizes for the input data
# print(predicted_sizes)

['LARGE']


In [35]:
import pickle

In [36]:
data = {
    'model': best_model,
    'OBJECT_AGE': [22.0],  # Replace with your input values
    'APOAPSIS': [35797.696],
    'PERIAPSIS': [33841.738],
    'MEAN_ANOMALY': [146.2900],
    'MEAN_MOTION': [1.038224],
    'INCLINATION': [12.1717],
    'PERIOD': [1386.984],
    'ECCENTRICITY': [0.023739]
}
with open('randomforestmodel.pkl', 'wb') as file:
    pickle.dump(data, file)

In [37]:
with open('randomforestmodel.pkl', 'rb') as file:
    data = pickle.load(file)

best_model= data["model"]
object_age = data["OBJECT_AGE"]
apoapsis = data["APOAPSIS"]
periapsis = data["PERIAPSIS"]
mean_anomaly= data["MEAN_ANOMALY"]
mean_motion= data["MEAN_MOTION"]
inclination= data["INCLINATION"]
period= data["PERIOD"]
eccentricity= data["ECCENTRICITY"]


In [38]:
y_pred = best_model.predict(X)
y_pred

array([2, 1, 3, ..., 3, 3, 3], dtype=int64)